In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.18.0"]])

(ns chp4
    (:refer-clojure :exclude [partial zero? + - * / ref compare =])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]))
(def render (comp ->infix simplify))

#'chp4/render

In [2]:
;; two general vector fields
(let-coordinates
 [[x y] R2-rect]
 (def e0 (+ (* (literal-manifold-function 'e0x R2-rect) d:dx)
            (* (literal-manifold-function 'e0y R2-rect) d:dy)))
 (def e1 (+ (* (literal-manifold-function 'e1x R2-rect) d:dx)
            (* (literal-manifold-function 'e1y R2-rect) d:dy))))

#'chp4/e1

In [3]:
(def e-vector-basis (down e0 e1))

#'chp4/e-vector-basis

In [4]:
(def e-dual-basis 
    (vector-basis->dual e-vector-basis R2-polar))

#'chp4/e-dual-basis

- The coefficients of the basis vectors $\mathsf{e}$, $\mathsf{c}_j^k s$, are associated with the polar coordinate system. 
- The coefficients of the dual one-forms $\tilde{\mathsf{e}}$, $\mathsf{d}_l^i s$, are associated with the polar coordinate system too. 
- $\mathsf{d}_l^i s$ are calculated from $\mathsf{c}_j^k s$ by matrix inversion.
- The duality is independent of the coordinates in which they are express.

In [5]:
(def R2-rect-point ((point R2-rect) (up 'x0 'y0)))
(render ((e-dual-basis e-vector-basis) R2-rect-point))

"up(down(1, 0), down(0, 1))"

- we can make a general vector field with this basis and then pick out the coefficients by applying the dual basis

In [6]:
(def v
    (* (up (literal-manifold-function 'b0 R2-rect)
           (literal-manifold-function 'b1 R2-rect))
       e-vector-basis))
(render ((e-dual-basis v) R2-rect-point))

"up(b0(up(x0, y0)), b1(up(x0, y0)))"

- Likewise, we can also make a general one form with the dual basis and then pick put the coefficients by applying the one form to the vector basis

In [7]:
(def omega
    (* (down (literal-manifold-function 'a_0 R2-rect)
             (literal-manifold-function 'a_1 R2-rect))
       e-dual-basis))
(render ((omega e-vector-basis) R2-rect-point))

"down(a₀(up(x0, y0)), a₁(up(x0, y0)))"

## 4.1 Change of Basis

In [8]:
(clojure.repl/source Jacobian)

(defn Jacobian
  "Returns the Jacobian of transition from `from-basis` to `to-basis`.

  The Jacobian is a structure of manifold functions. The outer index is the
  from-basis index, so this structure can be multiplied by tuple of component
  functions of a vector field relative to `from-basis` to get component
  functions for a vector field in `to-basis`."
  [to-basis from-basis]
  (s/mapr (basis->oneform-basis to-basis)
          (basis->vector-basis from-basis)))


nil

In [9]:
(def b-rect
    ((coordinate-system->oneform-basis R2-rect)
     (literal-vector-field 'b R2-rect)))

#'chp4/b-rect

In [10]:
(render (b-rect ((point R2-rect) (up 'x0 'y0))))

"up(b⁰(up(x0, y0)), b¹(up(x0, y0)))"

In [11]:
(def b-polar
    ((coordinate-system->oneform-basis R2-polar)
     (literal-vector-field 'b R2-rect)))

#'chp4/b-polar

In [12]:
(def b-polar-trans
    (* (Jacobian (coordinate-system->basis R2-polar)
                 (coordinate-system->basis R2-rect))
       b-rect))

#'chp4/b-polar-trans

In [13]:
(render (b-polar ((point R2-rect) (up 'x0 'y0))))

"up((x0 b⁰(up(x0, y0)) + y0 b¹(up(x0, y0))) / sqrt(x0² + y0²), (x0 b¹(up(x0, y0)) - y0 b⁰(up(x0, y0))) / (x0² + y0²))"

In [14]:
(render (b-polar-trans ((point R2-rect) (up 'x0 'y0))))

"up((x0 b⁰(up(x0, y0)) + y0 b¹(up(x0, y0))) / sqrt(x0² + y0²), (x0 b¹(up(x0, y0)) - y0 b⁰(up(x0, y0))) / (x0² + y0²))"

## 4.2 Rotation Basis

In [15]:
(* (rotate-z-matrix 'ϕ)
   (rotate-x-matrix 'θ)
   (rotate-z-matrix 'ψ))

#object[sicmutils.matrix.Matrix 0x7963b697 "[[(+ (* (cos ϕ) (cos ψ)) (* (- (sin ϕ)) (cos θ) (sin ψ))) (+ (* (cos ϕ) (- (sin ψ))) (* (- (sin ϕ)) (cos θ) (cos ψ))) (* (- (sin ϕ)) (- (sin θ)))] [(+ (* (sin ϕ) (cos ψ)) (* (cos ϕ) (cos θ) (sin ψ))) (+ (* (sin ϕ) (- (sin ψ))) (* (cos ϕ) (cos θ) (cos ψ))) (* (cos ϕ) (- (sin θ)))] [(* (sin θ) (sin ψ)) (* (sin θ) (cos ψ)) (cos θ)]]"]

In [16]:
(* (rotate-z-matrix 3)
   (rotate-x-matrix 2)
   (rotate-z-matrix 1)) ;; intrindix

#object[sicmutils.matrix.Matrix 0x2b1538ce "[[-0.4854784609636683 0.864780102737098 0.12832006020245673] [0.42291857174254777 0.1038465651516682 0.9001976297355174] [0.7651474012342926 0.49129549643388193 -0.4161468365471424]]"]

In [17]:
(* (rotate-z-matrix 1)
   (rotate-x-matrix 2)
   (rotate-z-matrix 3))

#object[sicmutils.matrix.Matrix 0x7fc32f1 "[[-0.4854784609636683 -0.42291857174254777 0.7651474012342926] [-0.864780102737098 0.1038465651516682 -0.49129549643388193] [0.12832006020245673 -0.9001976297355174 -0.4161468365471424]]"]

In [18]:
(render *1)

"matrix-by-rows(up(-0.4854784609636683, -0.42291857174254777, 0.7651474012342926), up(-0.864780102737098, 0.1038465651516682, -0.49129549643388193), up(0.12832006020245673, -0.9001976297355174, -0.4161468365471424))"

In [19]:
(* (rotate-z-matrix 'ϕ)
   (rotate-y-matrix 'θ)
   (rotate-z-matrix 'ψ))

#object[sicmutils.matrix.Matrix 0x22be2850 "[[(+ (* (cos ϕ) (cos θ) (cos ψ)) (* (- (sin ϕ)) (sin ψ))) (+ (* (cos ϕ) (cos θ) (- (sin ψ))) (* (- (sin ϕ)) (cos ψ))) (* (cos ϕ) (sin θ))] [(+ (* (sin ϕ) (cos θ) (cos ψ)) (* (cos ϕ) (sin ψ))) (+ (* (sin ϕ) (cos θ) (- (sin ψ))) (* (cos ϕ) (cos ψ))) (* (sin ϕ) (sin θ))] [(* (- (sin θ)) (cos ψ)) (* (- (sin θ)) (- (sin ψ))) (cos θ)]]"]

In [20]:
(render *1)

"matrix-by-rows(up(cos(ϕ) cos(θ) cos(ψ) - sin(ψ) sin(ϕ), - cos(ϕ) cos(θ) sin(ψ) - cos(ψ) sin(ϕ), cos(ϕ) sin(θ)), up(cos(θ) cos(ψ) sin(ϕ) + cos(ϕ) sin(ψ), - cos(θ) sin(ψ) sin(ϕ) + cos(ϕ) cos(ψ), sin(ϕ) sin(θ)), up(- cos(ψ) sin(θ), sin(ψ) sin(θ), cos(θ)))"

In [21]:
(* (rotate-z-matrix 3)
   (rotate-y-matrix 2)
   (rotate-z-matrix 1))

#object[sicmutils.matrix.Matrix 0x3868d3c0 "[[0.1038465651516682 -0.42291857174254777 -0.9001976297355174] [-0.864780102737098 -0.4854784609636683 0.12832006020245673] [-0.49129549643388193 0.7651474012342926 -0.4161468365471424]]"]

In [22]:
;; The euler angle rotation defined in the book.
(* (rotate-z-matrix 'ϕ)
   (rotate-x-matrix 'θ)
   (rotate-z-matrix 'ψ))

#object[sicmutils.matrix.Matrix 0x5c6fbcfe "[[(+ (* (cos ϕ) (cos ψ)) (* (- (sin ϕ)) (cos θ) (sin ψ))) (+ (* (cos ϕ) (- (sin ψ))) (* (- (sin ϕ)) (cos θ) (cos ψ))) (* (- (sin ϕ)) (- (sin θ)))] [(+ (* (sin ϕ) (cos ψ)) (* (cos ϕ) (cos θ) (sin ψ))) (+ (* (sin ϕ) (- (sin ψ))) (* (cos ϕ) (cos θ) (cos ψ))) (* (cos ϕ) (- (sin θ)))] [(* (sin θ) (sin ψ)) (* (sin θ) (cos ψ)) (cos θ)]]"]

In [23]:
(defn f
    [epsilon]
    (+ (up 'θ 'ϕ 'ψ) (* epsilon (up 'a 'b 'c))))

#'chp4/f

In [24]:
(def g (comp Euler->M f))

#'chp4/g

In [25]:
(render ((D g) 0))

"matrix-by-rows(up(a sin(ψ) sin(ϕ) sin(θ) - b cos(ϕ) cos(θ) sin(ψ) - c cos(θ) cos(ψ) sin(ϕ) - b cos(ψ) sin(ϕ) - c cos(ϕ) sin(ψ), a cos(ψ) sin(ϕ) sin(θ) - b cos(ϕ) cos(θ) cos(ψ) + c cos(θ) sin(ψ) sin(ϕ) + b sin(ψ) sin(ϕ) - c cos(ϕ) cos(ψ), a cos(θ) sin(ϕ) + b cos(ϕ) sin(θ)), up(- a cos(ϕ) sin(ψ) sin(θ) - b cos(θ) sin(ψ) sin(ϕ) + c cos(ϕ) cos(θ) cos(ψ) + b cos(ϕ) cos(ψ) - c sin(ψ) sin(ϕ), - a cos(ϕ) cos(ψ) sin(θ) - b cos(θ) cos(ψ) sin(ϕ) - c cos(ϕ) cos(θ) sin(ψ) - b cos(ϕ) sin(ψ) - c cos(ψ) sin(ϕ), - a cos(ϕ) cos(θ) + b sin(ϕ) sin(θ)), up(a cos(θ) sin(ψ) + c cos(ψ) sin(θ), a cos(θ) cos(ψ) - c sin(ψ) sin(θ), - a sin(θ)))"

### 4.3 Commutators

In [26]:
(def polar-vector-basis (coordinate-system->vector-basis R2-polar))
(def polar-dual-basis (coordinate-system->oneform-basis R2-polar))
(def f-rect (literal-manifold-function 'f-rect R2-rect))
(def e0 (* (up (literal-manifold-function 'e0x R2-rect)
               (literal-manifold-function 'e0y R2-rect))
           (coordinate-system->vector-basis R2-rect)))
(def e1 (* (up (literal-manifold-function 'e1x R2-rect)
                (literal-manifold-function 'e1y R2-rect))
           (coordinate-system->vector-basis R2-rect)))

#'chp4/e1

In [27]:
;; validate Eqn. 4.34
(simplify
((- ((commutator e0 e1) f-rect)
    (* (- (e0 (polar-dual-basis e1))
          (e1 (polar-dual-basis e0)))
       (polar-vector-basis f-rect)))
 R2-rect-point))

0

In [28]:
(let-coordinates
 [[x y z] R3-rect]
 (def Jz (- (* x d:dy) (* y d:dx)))
 (def Jx (- (* y d:dz) (* z d:dy)))
 (def Jy (- (* z d:dx) (* x d:dz))))

#'chp4/Jy

In [29]:
(def R3-rect-point ((point R3-rect) (up 'x0 'y0 'z0)))
(def g (literal-manifold-function 'g-rect R3-rect))

#'chp4/g

In [30]:
(render (((+ (commutator Jx Jy) Jz) g) R3-rect-point))

"0"

In [31]:
(let-coordinates
 [[θ ϕ ψ] Euler-angles]
 (def e_x (+ (* d:dθ (cos ϕ))
             (* d:dϕ -1 (/ (* (sin ϕ) (cos θ)) (sin θ)))
             (* d:dψ (/ (sin ϕ) (sin θ)))))
 (def e_y (+ (* d:dθ (sin ϕ))
             (* d:dϕ (/ (* (cos ϕ) (cos θ)) (sin θ)))
             (* d:dψ -1 (/ (cos ϕ) (sin θ)))))
 (def e_z d:dϕ))

#'chp4/e_z

In [32]:
(def SO3-point ((point Euler-angles) (up 'θ 'ϕ 'ψ)))
(def f (literal-manifold-function 'f-Euler Euler-angles))

#'chp4/f

In [73]:
(render (((+ (commutator e_x e_y) e_z) f) SO3-point))

"0"

In [33]:
(render (((+ (commutator e_y e_z) e_x) f) SO3-point))

"0"

In [34]:
(render (((+ (commutator e_z e_x) e_y) f) SO3-point))

"0"

- here we consider the evolution of a loop with sides made from the integral curves of two vector fields $Jx$ and $Jz$ at the vector $(0, 0, 1)$ in $R^3$. 

In [68]:
(def after-Jx-seq
    (((exp (* 't Jx)) (chart R3-rect))
     ((point R3-rect) (up 0 0 1))))
(def after-Jx
    (reduce + (take 3 after-Jx-seq)))

#'chp4/after-Jx

In [69]:
(def after-Jz-seq
    (((exp (* 't Jz)) (chart R3-rect))
     ((point R3-rect) after-Jx)))
(def after-Jz
    (reduce + (take 3 after-Jz-seq)))

#'chp4/after-Jz

In [70]:
(def after-inv-Jx-seq
    (((exp (* 't (- Jx))) (chart R3-rect))
     ((point R3-rect) after-Jz)))
(def after-inv-Jx
    (reduce + (take 3 after-inv-Jx-seq)))

#'chp4/after-inv-Jx

In [71]:
(def after-inv-Jz-seq
    (((exp (* 't (- Jz))) (chart R3-rect))
     ((point R3-rect) after-inv-Jx)))
(def after-inv-Jz
    (reduce + (take 3 after-inv-Jz-seq)))

#'chp4/after-inv-Jz

In [72]:
(render after-inv-Jz-seq)

"up(t², -1/4 t⁵ + 1/2 t³, -1/4 t⁴ + 1) + up(-1/4 t⁶ + 1/2 t⁴, - t³, 0) + up(-1/2 t⁴, 1/8 t⁷ -1/4 t⁵, 0) + up(1/24 t⁸ -1/12 t⁶, 1/6 t⁵, 0) + ..."

- here we consider the evolution of a loop with sides made from the integral curves of two vector fields $ex$ and $ez$ at the vector $(1, 0, 0)$ in $SO(3)$. 

In [127]:
(def t 0.1)
(def euler-angles-cood (up (/ pi 2) 0 0))
euler-angles-cood

(up 1.5707963267948966 0 0)

In [138]:
(def after-ex-seq
    (((exp (* t e_x)) (chart Euler-angles))
     ((point Euler-angles) euler-angles-cood)))
(def after-ex
    (reduce + (take 7 after-ex-seq)))
after-ex

(up 1.6791852497670348 0 0)

In [139]:
(def after-ez-seq
    (((exp (* t e_z)) (chart Euler-angles))
     ((point Euler-angles) after-ex)))
(def after-ez
    (reduce + (take 7 after-ez-seq)))
after-ez

(up 1.6791852497670348 0.1 0.0)

In [140]:
(def after-inv-ex-seq
    (((exp (* t (- e_x))) (chart Euler-angles))
     ((point Euler-angles) after-ez)))
(def after-inv-ex
    (reduce + (take 7 after-inv-ex-seq)))
after-inv-ex

(up 1.570472999881411 0.09883565859924028 -0.00990916635880347)

In [141]:
(def after-inv-ez-seq
    (((exp (* t (- e_z))) (chart Euler-angles))
     ((point Euler-angles) after-inv-ex)))
(def after-inv-ez
    (reduce + (take 7 after-inv-ez-seq)))
after-inv-ez

(up 1.570472999881411 -0.0011643414007597164 -0.00990916635880347)

In [122]:
(render after-inv-ez)

"up(1.570796326561851, -1.0015686259957218E-9, -9.999983325480414E-7)"

In [142]:
;; z first
(def after-ez-seq-2
    (((exp (* t e_z)) (chart Euler-angles))
     ((point Euler-angles) euler-angles-cood)))
(def after-ez-2
    (reduce + (take 7 after-ez-seq-2)))
after-ez-2

(up 1.5707963267948966 0.1 0.0)

In [144]:
;; then x 
(def after-ex-seq-2
    (((exp (* t e_x)) (chart Euler-angles))
     ((point Euler-angles) after-ez-2)))
(def after-ex-2
    (reduce + (take 7 after-ex-seq-2)))
after-ex-2

(up 1.67859728164069 0.10099825524795271 0.010066202125322365)